# LUMEN DataSci 2021

## Data loading

### Imports

In [1]:
import pandas as pd
import numpy as np

### Dataset

In [2]:
dataset = pd.read_csv('LUMEN_DS.csv', sep='|', quotechar='"', encoding='UTF-16LE')
dataset.shape

/home/msmetko/Projects/lumen/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (16,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(1294962, 33)

## Data cleaning

In [3]:
# List of all the features
for feature_name in dataset.keys():
    print(feature_name)

Manufacturing Region
Manufacturing Location Code
Intercompany
CustomerID
Customer industry
Customer Region
Customer First Invoice Date
Top Customer Group
Item Code
Product family
Product group
Price last modified date in the ERP
Born on date
Make vs Buy
Sales Channel - Internal
Sales Channel - External
Sales Channel - Grouping
Invoice Date
Invoice #
Invoice Line #
Order Date
Order #
Order Line #
Invoiced qty (shipped)
Ordered qty
Invoiced price
Invoiced price (TX)
Cost of part
Material cost of part
Labor cost of part
Overhead cost of part
GM%
# of unique products on a quote


In [4]:
# Drop all useless features
dataset.drop(
    ['CustomerID', 'Item Code', 'Invoice #', 'Order #', 'Invoiced price (TX)'],
    axis=1, inplace=True)

In [5]:
# Share of NaN's per feature
na_share = dataset.isna().mean()
na_share

Manufacturing Region                   0.031282
Manufacturing Location Code            0.030857
Intercompany                           0.000000
Customer industry                      0.000000
Customer Region                        0.000080
Customer First Invoice Date            0.000000
Top Customer Group                     0.000000
Product family                         0.000000
Product group                          0.130243
Price last modified date in the ERP    0.738107
Born on date                           0.048769
Make vs Buy                            0.014285
Sales Channel - Internal               0.130760
Sales Channel - External               0.130776
Sales Channel - Grouping               0.999984
Invoice Date                           0.000000
Invoice Line #                         0.000000
Order Date                             0.000000
Order Line #                           0.040761
Invoiced qty (shipped)                 0.000000
Ordered qty                            0

`'Price last modified date in the ERP'` has a lot of missing values **(73.8%)**, since a lot of prices never changed. Therefore, we put the oldest change date to all the `NaN`s

In [6]:
column = 'Price last modified date in the ERP'
last_modified = dataset[column]
oldest = last_modified[last_modified.notna()].min()
dataset.loc[last_modified.isna(), column] = oldest
assert not dataset[column].isna().any()

Drop all the features (column) which have more than 10% of `Nan`'s

**TODO: impute them using regressor**

In [7]:
dataset.drop(na_share[na_share > 0.1].index, axis=1, inplace=True)
na_share = dataset.isna().mean()
na_share

Manufacturing Region               0.031282
Manufacturing Location Code        0.030857
Intercompany                       0.000000
Customer industry                  0.000000
Customer Region                    0.000080
Customer First Invoice Date        0.000000
Top Customer Group                 0.000000
Product family                     0.000000
Born on date                       0.048769
Make vs Buy                        0.014285
Invoice Date                       0.000000
Invoice Line #                     0.000000
Order Date                         0.000000
Order Line #                       0.040761
Invoiced qty (shipped)             0.000000
Ordered qty                        0.040761
Invoiced price                     0.006691
Cost of part                       0.018346
GM%                                0.090444
# of unique products on a quote    0.040777
dtype: float64

## Drop all rows with NaN's
**TODO: impute then using regressor**

In [11]:

dataset.dropna(inplace=True)
dataset.shape

(1102022, 20)

In [9]:
#import plotly.express as ex
#df = dataset.isna().mean()
#ex.histogram(df, nbins=1000)